In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = np.genfromtxt("../data/tng-100-3/cutdata/central_id.txt")

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_id.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [stars] #gas, dm, 
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [2]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    #gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    #dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    #dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [3]:
group_cat = physics.properties.galaxy_radius(group_cat, basePath)
group_cat = physics.properties.center_halo(stars, N, group_cat)
for particle in particle_lists:
    particle = physics.properties.relative_pos_radius(particle, N, group_cat)

for i in range(N):
    max_rad = group_cat["SubhaloGalaxyRad"][i]
    for particle in particle_lists:
        particle[i] = particle[i][particle[i]["r"] < max_rad]

print("Calculating stellar masses")
group_cat = physics.properties.total_mass(stars, N, group_cat)
print("Calculating subhalo velocity")
stars, group_cat = physics.properties.subhalo_velocity(stars, N, group_cat)
print("Calculating relative velocities")
stars = physics.properties.relative_velocities(stars, N, group_cat)
print("Calculating half mass radius")
group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")
print("Calculating maximum angular momentum")
group_cat = physics.properties.max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
print("Rotating subhalos")
stars = physics.geometry.rotate_coordinates(stars, N, rot_vector)

#End timer
end = timer()

Calculating stellar masses
Calculating subhalo velocity
Calculating relative velocities
Calculating half mass radius
Calculating maximum angular momentum
Rotating subhalos


In [4]:
print("Galaxy radius")
print(group_cat["SubhaloGalaxyRad"])

Galaxy radius
0      154.633
1       149.16
2      118.384
3      135.639
4      124.606
        ...   
99     51.2713
100    51.4321
101    45.9509
102    39.8238
103    39.4701
Name: SubhaloGalaxyRad, Length: 104, dtype: object


In [5]:
print("Masses check")
for i in range(N):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])

Masses check
Group catalogue:  80.1757  Snapshot:  54.72526931762695
Group catalogue:  90.369896  Snapshot:  58.46538543701172
Group catalogue:  43.96099  Snapshot:  30.320581436157227
Group catalogue:  42.83398  Snapshot:  30.461362838745117
Group catalogue:  56.558517  Snapshot:  43.7795524597168
Group catalogue:  87.63501  Snapshot:  70.92870330810547
Group catalogue:  71.43518  Snapshot:  56.552490234375
Group catalogue:  31.020468  Snapshot:  24.84549331665039
Group catalogue:  61.905563  Snapshot:  46.23890686035156
Group catalogue:  61.568096  Snapshot:  50.89623260498047
Group catalogue:  51.169712  Snapshot:  40.30483627319336
Group catalogue:  18.015245  Snapshot:  14.132622718811035
Group catalogue:  16.47061  Snapshot:  13.221027374267578
Group catalogue:  38.428806  Snapshot:  31.063730239868164
Group catalogue:  32.70103  Snapshot:  24.75693702697754
Group catalogue:  26.348753  Snapshot:  18.92565155029297
Group catalogue:  25.611813  Snapshot:  19.45177459716797
Group c

In [6]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelX"][i], group_cat["SubhaloVelY"][i], group_cat["SubhaloVelZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-171.23763665295385 511.42953563603055 -22.79878854361744
[-175.13785    81.367256   -8.339443]
208.42888417637573 -144.33385048683883 91.88748882556843
[279.23257  -99.84879   82.072495]
103.81072984261439 72.9156542950619 224.41707709524943
[174.64732  57.31744 170.41402]


In [7]:
print("Halfmass radius check")
for i in range(N): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  64.70497  Snapshot:  31.57238065633738
Group catalogue:  79.047134  Snapshot:  36.121992901430765
Group catalogue:  33.925236  Snapshot:  12.658515597823898
Group catalogue:  40.62605  Snapshot:  15.872608862831003
Group catalogue:  22.51544  Snapshot:  10.342553241262525
Group catalogue:  35.24243  Snapshot:  25.357411437842035
Group catalogue:  39.33135  Snapshot:  27.91179017846851
Group catalogue:  25.502897  Snapshot:  15.98410238078073
Group catalogue:  42.428154  Snapshot:  25.435664895404276
Group catalogue:  27.368404  Snapshot:  20.199075745338785
Group catalogue:  28.622637  Snapshot:  18.6557231777787
Group catalogue:  20.585445  Snapshot:  14.35802226543256
Group catalogue:  15.178419  Snapshot:  10.368649653735417
Group catalogue:  21.49089  Snapshot:  15.39184360570783
Group catalogue:  13.425839  Snapshot:  8.10292985488646
Group catalogue:  39.58687  Snapshot:  20.970284415016593
Group catalogue:  36.317265  Snapshot:  26.3364187

In [8]:

#Save all fields for particles in one subhalo for later inspection
g = 10
stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(int(indices[g])) +"_stars.pkl")
print(indices[g])

3229.0


In [9]:
#Save group catalogue
#group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [10]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 104 Subhalos: 
32 Seconds


In [11]:
import check_rotation
check_rotation.test(group_cat, 'tng-100-3', 'simple')

TypeError: object of type 'numpy.int32' has no len()

In [12]:
latest_id = 3229
most_late_index = group_cat[group_cat["id"] == latest_id].index.values.astype(int)[0]
most_late = stars[most_late_index]